# Object Oriented Programming in R

## Environments as objects

Inspired by RC, R6 and _proto_ classes (objects) we present how we can define our own object/class system. 

We start by defining objects. For that we use standard R environment. 

In [1]:
objectCreator = function(vars=list()) {
    envir = new.env()
    for (vn in names(vars)) {
        assign(vn, vars[[vn]], envir = envir)
        ## Alternative way
        ## env[[vn]] <- vars[[vn]]
    }
    envir
}

complexNum <- objectCreator(
    vars=list(
        x=1, 
        y=2
    )
)

print(complexNum$x)
print(complexNum$y)

print(ls.str(complexNum))


[1] 1
[1] 2
x :  num 1
y :  num 2


### Adding Methods

What we have done so far could be easyly achived by using list. But if we wnat to have methods, is where environment comes handy. Lets try.

In [2]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        re=function() {
            # This won't work
            x
        }
    )
)

print(cn$re())

ERROR: Error in cn$re(): object 'x' not found


<environment: R_GlobalEnv>


The problem is the environment of this function. 

In [10]:
print(environment(cn$re))

<environment: R_GlobalEnv>


So let assigin to environment of the function the environment of the object.

In [3]:
objectCreator = function(vars=list()) {
    envir = new.env()
    for (vn in names(vars)) {
        envir[[vn]] <- vars[[vn]]
        if (is.function(envir[[vn]])) {
          environment(envir[[vn]]) <- envir
        }
    }
    envir
}

cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        re=function() {
            x
        }
    )
)

print(cn$re())


[1] 1


This was a getter. Now time of setting an x. Let's project out complex number to x asis.

In [5]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        projectX=function() {
            # This won't work
            y <- 0
        }
    )
)

cn$projectX()
print(cn$y)

[1] 2


In [8]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        projectX=function() {
            # This won't work
            y <- 0
            environment()
        }
    )
)

env <- cn$projectX()
print(env)
print(parent.env(env))
print(cn)

<environment: 0x27e62a0>
<environment: 0x185f800>
<environment: 0x185f800>


In [9]:
cn <- objectCreator(
    vars=list(
        x=1, 
        y=2,
        projectX=function() {
            y <<- 0
        }
    )
)
cn$projectX()
print(cn$y)

[1] 0


## Clasess

Now it is time for classes. Class is a set of objects that share the same attributes and method. Something like:
Class of complex number is a set of _things_ that have an attribute _x_, _y_, and method _re_, _im_ etc......

In order to implement it we need two functions. First a _function_ that transform a recipe into a function that later can produce for us objects that agree with this recipe (instances of this class).

In [18]:
classGenerator <- function(Class) {
    function() objectCreator(Class)
}

And that is it. Let's see it in action.

In [22]:
ComplexNumber <- classGenerator(list(
    x=0,
    y=0,
    re=function() x,
    im=function() y,
    projectX=function() {y <<- 0}
))
   
cn1 <- classGenerator(ComplexNumber)
cn2 <- classGenerator(ComplexNumber)
print(cn1$x)
print(cn2$x)

[1] 0
[1] 0


That was a bit silly. 

## S3 Class